In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import pickle
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import os
import numpy as np
import string

import pandas as pd
from nltk.corpus import stopwords
import argparse

default_working_dir = "/content/drive/MyDrive/n2c2/trainingdata_v3/trainingdata_v3/"
parser = argparse.ArgumentParser();
parser.add_argument('--max_seq_length', type=int, default=128);
parser.add_argument('--working_dir', type=str, default= default_working_dir)
parser.add_argument('-f')
args = parser.parse_args();

#global variable
Entity_Count = 0;
Entities_Captured = 0;

def Parse_Annotation(filepath = ""):
    Annotation = open(filepath, "r");
    Annotation = Annotation.readlines();
    global Entity_Count;

    Ann_List = [];
    remove_list = list(string.punctuation);
    remove_list.remove(";");
    for Ann in Annotation:
        for punct in remove_list:
            Ann = Ann.replace(punct, " ");
        Ann = Ann.replace(".", "");

        if Ann[0] == "T":
            Entity_Count+= 1;
            Ann = nltk.tokenize.word_tokenize(Ann);
            if len(Ann) > 5 and Ann.count(";")>0:
                Ann = [int(Ann[2]), Ann[1]] + Ann[7:];
                if len(Ann) > 3:
                    BAnn = tuple(Ann[:3]);
                    Ann_List.append(BAnn);
                    Term_index = Ann[0] + 1;
                    for IAnn in Ann[3:]:
                        tag = "I" + Ann[1];
                        IAnn = (Term_index, tag, IAnn);
                        Term_index = Term_index + 1;
                        Ann_List.append(IAnn);
                else:
                    Ann = tuple(Ann);
                    Ann_List.append(Ann);
            else:
                Ann = [int(Ann[2]), Ann[1]] + Ann[4:];
                if len(Ann) > 3:
                    BAnn = tuple(Ann[:3]);
                    Ann_List.append(BAnn);
                    Term_index = Ann[0] + 1;
                    for IAnn in Ann[3:]:
                        tag = "I" + Ann[1];
                        IAnn = (Term_index, tag, IAnn);
                        Term_index = Term_index + 1;
                        Ann_List.append(IAnn);
                else:
                    Ann = tuple(Ann);
                    Ann_List.append(Ann);

    Term_index = []
    Term_dict = {};
    [Term_index.append(x[0]) for x in Ann_List]
    Term_index.sort();
    for Terms in Term_index:
        Term_dict[Terms] = None;
    for tuples in Ann_List:
        Term_dict[tuples[0]] = tuples[1:];

    Ann_List = list(Term_dict.values());
    Ann_len = len(Ann_List);
    #Entity_Count += Ann_len;
    return Ann_List;

def Tokenize_EHR(file_path = ""):
    EHR_Tokenized = [];
    EHR_txt = open(file_path, 'r').read();
    # remove specific characters from text
    punctuation = list(string.punctuation);
    punctuation.remove('.');
    # punctuation.remove('?');
    # punctuation.remove('!');
    for punct in punctuation:
        EHR_txt = EHR_txt.replace(punct, " ")
    #Split text by paragraph
    EHR_Sections = EHR_txt.split("\n\n\n")

    #split section by sentances(sentace indicators: ., ?, !)
    for section in EHR_Sections:
        if(section == ""): continue;
        else:
            # sentances = section.split(".");
            # sentances = section.split("!");
            # sentances = section.split("?");
            sentances = nltk.sent_tokenize(section);
            # split sentances into tokens
            for sent in sentances:
                sent = sent.replace(".", " ")
                # sent = sent.replace("!", " ")
                # sent = sent.replace("?", " ")
                sent_tokens = nltk.word_tokenize(sent);
                EHR_Tokenized.append(sent_tokens);
            # for sent in sentances:
            #     sent = sent.replace("?", " ")
            #     sent_tokens = nltk.word_tokenize(sent);
            #     EHR_Tokenized.append(sent_tokens);
            # for sent in sentances:
            #     sent = sent.replace("!", " ")
            #     sent_tokens = nltk.word_tokenize(sent);
            #     EHR_Tokenized.append(sent_tokens);


    return EHR_Tokenized;

def Generate_cmed_data(EHR_data_dir = "", EHR_fileID = "", write_file = False, max_seq_length = 128):

    Tags = {"Disposition":("B-Disposition"), "IDisposition":("I-Disposition"), "NoDisposition":("B-NoDisposition"),
           "INoDisposition":("I-NoDisposition"), "Undetermined":("B-Undetermined"), "IUndetermined":("I-Undetermined"), "Outside":{"O"}};


    EHR_Ann_filepath = EHR_data_dir + EHR_fileID + ".ann";
    EHR_doc_filepath = EHR_data_dir + EHR_fileID + ".txt";
    EHR_Tagged_filepath = EHR_data_dir + EHR_fileID + "_Tagged.txt";

    EHR_Tokens = Tokenize_EHR(EHR_doc_filepath);
    Parsed_Annotation = Parse_Annotation(EHR_Ann_filepath);
    Parsed_Annotation.append(("[END]", "[END]"));

    Tagged_EHR_List = [];

    if(write_file) :Tagged_EHR_File = open(EHR_Tagged_filepath, "w");
    Annotation_index = 0;

    stop_words = set(stopwords.words('english'));
    removal_list = list(stop_words);

    EHR_ID_Token = "RecordID" + str(EHR_fileID);
    Tagged_EHR_List.append((EHR_ID_Token, 'O'));
    Tagged_EHR_List.append(("",""));

    global Entities_Captured;
    for sents in EHR_Tokens:
        seq_index = 0;
        if(len(sents) == 1) and (Parsed_Annotation[Annotation_index][1] not in sents): continue;
        for words in sents:
           #Make specific changes relating to dataset here
           if Parsed_Annotation[Annotation_index][1] in removal_list:removal_list.remove(Parsed_Annotation[Annotation_index][1])
           elif(words in removal_list): continue;
           elif("___" in words): continue;
           #elif Parsed_Annotation[Annotation_index][1] in removal_list:
           #    Annotation_index+= 1;
           #    Entities_Captured += 1;

           if(Parsed_Annotation[Annotation_index][1] in words) or (len(words) > 3 and words in Parsed_Annotation[Annotation_index][1]):
               Entities_Captured += 1;
               tag = Tags[Parsed_Annotation[Annotation_index][0]];
               Tagged_EHR_List.append((words, tag))
               seq_index += 1;
               if(seq_index == max_seq_length):
                   Tagged_EHR_List.append(("", ""));
                   seq_index = 0;
               if(write_file): Tagged_EHR_File.write(words + " " + tag + "\n");
               Annotation_index = Annotation_index + 1;

           else:
               Tagged_EHR_List.append((words, 'O'));
               seq_index += 1;
               if(seq_index == max_seq_length):
                   Tagged_EHR_List.append(("", ""));
                   seq_index = 0;
               if(write_file):Tagged_EHR_File.write(words + " " + '0' + "\n");

        if(seq_index>0):Tagged_EHR_List.append(("", ""));
        if(write_file):Tagged_EHR_File.write("\n");

    if(write_file):Tagged_EHR_File.close();

    return np.array(Tagged_EHR_List).transpose();

def main():
    global Entity_Count;
    global Entities_Captured;

    Working_dir = "/content/drive/MyDrive/n2c2/trainingdata_v3/trainingdata_v3/";
    Working_dir = args.working_dir;
    max_seq_length = args.max_seq_length;
    Training_data_dir = Working_dir + "train/";
    Eval_data_dir = Working_dir + "dev/";

    Training_files_list = os.listdir(Training_data_dir);
    Eval_files_list = os.listdir(Eval_data_dir);

    #Test case
    #Generate_cmed_data(EHR_data_dir=Training_data_dir, EHR_fileID="336-01", write_file=True)

    #Build training data for Biobert model
    cmed_data_dict = {"Tokens":[], "Tags":[]};
    Data_files_dict = {};
    for files in Training_files_list:
       Data_files_dict[files[:6]] = None;
    for IDs in Data_files_dict:
        if IDs == '.DS_St':
         continue
        [tokens, tags] = Generate_cmed_data(EHR_data_dir=Training_data_dir, EHR_fileID=IDs, write_file = False, max_seq_length=max_seq_length);

        #if Entity_Count != Entities_Captured:
        #    print("Entities Not Captured");
        #    print(Entity_Count);
        #    print(Entities_Captured);
        #    print();

        cmed_data_dict["Tokens"].extend(tokens);
        cmed_data_dict["Tags"].extend(tags);
    cmed_data = pd.DataFrame(cmed_data_dict);
    train_data_filename = Working_dir + "train_dev.tsv";
    cmed_data.to_csv(train_data_filename, sep="\t",header=False, index = False );

    print("-------Training Entities Summary--------")
    print("Total Entites:", Entity_Count);
    print("Entities Captured:", Entities_Captured);
    print("Percent Captured:", np.round(Entities_Captured/Entity_Count, 2));
    Entity_Count = 0;
    Entities_Captured = 0;

    #Build eval data for Biobert model
    cmed_data_dict = {"Tokens": [], "Tags": []};
    Data_files_dict = {};
    for files in Eval_files_list:
       Data_files_dict[files[:6]] = None;
    for IDs in Data_files_dict:
        if IDs == '.DS_St':
          continue
        [tokens, tags] = Generate_cmed_data(EHR_data_dir=Eval_data_dir, EHR_fileID=IDs, write_file = False);
        cmed_data_dict["Tokens"].extend(tokens);
        cmed_data_dict["Tags"].extend(tags);

    cmed_data = pd.DataFrame(cmed_data_dict);
    eval_data_filename = Working_dir + "devel.tsv";
    cmed_data.to_csv(eval_data_filename, sep="\t",header=False, index = False );

    test_data_filename = Working_dir + "test.tsv";
    cmed_data.to_csv(test_data_filename, sep="\t",header=False, index = False );

    print("-------Eval Entities Summary--------")
    print("Total Entites:", Entity_Count);
    print("Entities Captured:", Entities_Captured);
    print("Percent Captured:", np.round(Entities_Captured/Entity_Count, 2));
    Entity_Count = 0;
    Entities_Captured = 0;

    return 0;


if __name__ == '__main__':
    main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
-------Training Entities Summary--------
Total Entites: 6196
Entities Captured: 7207
Percent Captured: 1.16
-------Eval Entities Summary--------
Total Entites: 1033
Entities Captured: 1153
Percent Captured: 1.12
